In [ ]:
# 导入所需库并读取训练文件
import seaborn as sns
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
import itertools
import warnings
# 读取数据
df = pd.read_csv("./student_data.csv")

# 使用混合模型提高预测准确率
## 贝叶斯混合模型
该模型分类精度很高，但是需要数据集较大，且数据集中的每个类别的样本量要均匀分布。此数据集中C Programme的样本量较少，可能会影响模型的准确率。并且grade 3数量也很少，并且绝大多数以C programme为主。

## knn分类模型
该模型分类精度较差，但是对于特殊类别，比如说C Programme和grade 3的样本量较少的类别，knn模型的分类精度较高。我们将使用knn模型进行分类。我们将使用投票模型的方式为数据集中数量较少的特殊组创建pca+kmean模型进行分类。接下来我们将要训练knn模型。

In [ ]:

# 提取特征和目标变量
X = df.drop(columns=["Programme"])
y = df["Programme"]

# 创建PCA + 分类器的管道
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('classifier', RandomForestClassifier(random_state=42))
])

# 定义参数网格
param_grid = {
    'pca__n_components': [1, 2, 3, 4, 5, 6, 7],
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 5, 10, 15]
}

# 使用网格搜索查找最佳参数
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# 打印最佳参数和分数
print(f"最佳参数: {grid_search.best_params_}")
print(f"最佳交叉验证分数: {grid_search.best_score_:.4f}")

# 使用最佳参数进行PCA
best_components = grid_search.best_params_['pca__n_components']
best_pipeline = grid_search.best_estimator_

# 获取PCA结果用于可视化
pca = best_pipeline.named_steps['pca']
X_scaled = best_pipeline.named_steps['scaler'].transform(X)
X_pca = pca.transform(X_scaled)
